# Corrupted Data Recovery

This is a trivial implementation of Corrupted Data Recovery as specified in Recover Corrupted Data in Sensor Networks: A Matrix Completion Solution published in vol 16 of IEEE Transaction On Mobile Computing
Link: http://ieeexplore.ieee.org/document/7526433

DataSet: http://db.csail.mit.edu/labdata/labdata.html

### Imports

In [20]:
from  fancyimpute import NuclearNormMinimization, KNN , SoftImpute
import numpy as np 

### Data preprocessing

In [21]:
a= [[]]
file = open("data.txt","r")
x = file. readlines()
for line in x:             
    fields = []
    fields = line.split()
    a.append(fields)
a = a[1:]
print(len(a))
for i in range(len(a)-1,0,-1):
    if(len(a[i])<8):
        del a[i]
print(len(a))
b = np.zeros((len(a),3))
for i in range(3):
    for j in range(len(a)):
        b[j,i] = float(a[j][2+i])

print b

2313682
2219803
[[2.00000e+00 1.00000e+00 1.22153e+02]
 [3.00000e+00 1.00000e+00 1.99884e+01]
 [1.10000e+01 1.00000e+00 1.93024e+01]
 ...
 [6.24240e+04 5.00000e+00 2.41436e+01]
 [6.24260e+04 5.00000e+00 2.41240e+01]
 [6.24270e+04 5.00000e+00 2.41044e+01]]


In [22]:
print(b.shape)

(2219803, 3)


In [23]:
N = int(max(b[:,1]))
T = int(max(b[:,0]))
T = int(0.10*T)
print N
print T
M = np.empty((N,T))
M[:] = np.nan
print M

54
6549
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


In [24]:
for i in range(len(b)):
    if b[i][0]<=T:
        M[int(b[i][1])-1][int(b[i][0])-1] = b[i][2]    
print M

[[     nan 122.153   19.9884 ...  20.9096      nan      nan]
 [     nan  89.5488      nan ...  20.9096  20.89        nan]
 [     nan  20.204  122.153  ...  21.9288      nan  21.9288]
 ...
 [     nan      nan  18.3028 ...      nan  20.0276  22.0828]
 [     nan      nan      nan ...      nan      nan  19.714 ]
 [     nan 122.016  122.153  ...      nan  19.4984  19.4788]]


In [25]:
import math
count = 0
for i in range(len(M)):
    for x in M[i]:
        if math.isnan(x):
            count =  count +1
print count

136829


### Initialization

N is the number of sensors.<br>
T is the number of Time slots.<br>
Y is One Hop Matrix. <br> 
W is Weight matrix, where Wji represents how strong is the relation ship between Timeslots j and i<br>
x_structual_faults is a list representing the row numbers with structral faults<br>
y_structural_faults is a list representing the Column numbers with structral faults

In [26]:
file = open('connectivity.txt','r')
a= [[]]
for line in file:
    temp = []
    temp = line.split()
    if len(temp) is 3 and int(temp[0]) > 0 and int(temp[0]) <= 54 and int(temp[1]) > 0 and int(temp[1])<=54: 
        temp[0] = int(temp[0])
        temp[1] = int(temp[1])
        if float(temp[2]) > 0.5:
            temp[2] = 1
        else:
            temp[2] = 0
        a.append(temp)
a = a[1:]
a = np.array(a)


In [27]:
'''
N is the number of sensors.
T is the number of Time slots
M is the sansory Matrix 
Y is One Hop Matrix 
w is weight matrix
x_structual_faults is a list representing the row numbers with structral faults
y_structural_faults is a list representing the Column numbers with structral faults
Y is one hop matrix
w is weight matrix
'''
Y = np.zeros((N,N))
for x in a:
    Y[x[0]-1][x[1]-1] = x[2]
print Y
w = np.empty((T,T))
#w = np.array([[1,2,3,4,5],[1,5,7,6,3],[1,8,6,2,9],[7,4,2,6,8],[3,4,2,8,1]])
w = np.random.rand(T,T)
print w
x_structural_faults = np.random.randint(1,N-1,size = 5)
print x_structural_faults
x_structural_faults= -np.sort(-x_structural_faults)
y_structural_faults =  np.random.randint(1,T-1,size = 5)
print 'hello'
y_structural_faults = -np.sort(-y_structural_faults)
print y_structural_faults

[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]
[[0.29147938 0.5266544  0.59554076 ... 0.8901134  0.53913884 0.0418196 ]
 [0.82544209 0.02420024 0.34462561 ... 0.3041763  0.71631948 0.14223306]
 [0.28863371 0.52580167 0.42614991 ... 0.34405556 0.58706737 0.12733367]
 ...
 [0.6142622  0.90347016 0.26103502 ... 0.47767324 0.80460277 0.48210031]
 [0.73060208 0.28189131 0.71967627 ... 0.70431693 0.37366273 0.58160473]
 [0.16472452 0.68781054 0.07804674 ... 0.0291432  0.55388796 0.129807  ]]
[15 22 49 27 49]
hello
[1900 1835 1827 1217  448]


### First Phase 

#### Step 1 : Removing of rows and columns with Structural Faults 

In [28]:
M_old = M
for i in x_structural_faults:
    M = np.delete(M, (i), axis=0)
for i in y_structural_faults:
    M = np.delete(M, (i), axis=1)
print "orignal M shape"
print M_old.shape
print "M without structural faults shape"
print M.shape

orignal M shape
(54, 6549)
M without structural faults shape
(49, 6544)


#### Step 2 : Removing of Empty cells 

In [29]:
'''
Removes the empty cells represented by NaNs(Null Values)
'''
#M = NuclearNormMinimization().complete(M)
#M = KNN(k=3).complete(M)
M =  SoftImpute().complete(M)

[SoftImpute] Max Singular Value of X_init = 13404.219757
[SoftImpute] Iter 1: observed MAE=2.403761 rank=49
[SoftImpute] Iter 2: observed MAE=2.397293 rank=49
[SoftImpute] Iter 3: observed MAE=2.377285 rank=48
[SoftImpute] Iter 4: observed MAE=2.349452 rank=48
[SoftImpute] Iter 5: observed MAE=2.323554 rank=47
[SoftImpute] Iter 6: observed MAE=2.295661 rank=46
[SoftImpute] Iter 7: observed MAE=2.271014 rank=46
[SoftImpute] Iter 8: observed MAE=2.250031 rank=46
[SoftImpute] Iter 9: observed MAE=2.231102 rank=46
[SoftImpute] Iter 10: observed MAE=2.213772 rank=46
[SoftImpute] Iter 11: observed MAE=2.197916 rank=46
[SoftImpute] Iter 12: observed MAE=2.183401 rank=46
[SoftImpute] Iter 13: observed MAE=2.170213 rank=46
[SoftImpute] Iter 14: observed MAE=2.158300 rank=46
[SoftImpute] Iter 15: observed MAE=2.147601 rank=46
[SoftImpute] Iter 16: observed MAE=2.138029 rank=46
[SoftImpute] Iter 17: observed MAE=2.129511 rank=46
[SoftImpute] Iter 18: observed MAE=2.121944 rank=46
[SoftImpute] Ite

#### Step 3 : Updating the original Sensory Matrix 

In [30]:
print M.shape
print M

(49, 6544)
[[ 20.98807397 122.153       19.9884     ...  20.9096      17.83251378
   17.54036854]
 [ 12.57807911  89.5488      29.54739866 ...  20.9096      20.89
   18.13904648]
 [ 18.62212417  20.204      122.153      ...  21.9288      18.12322858
   21.9288    ]
 ...
 [ 13.97419419  33.14470172  18.3028     ...  16.13006729  20.0276
   22.0828    ]
 [ 18.04571103  30.16374397  28.98459688 ...  15.26862217  15.52329115
   19.714     ]
 [ 18.23895245 122.016      122.153      ...  16.02093506  19.4984
   19.4788    ]]


In [31]:
'''
Updating the orignal Sensory Matrix by the values for empty positions
'''
i1=j1=0
for i in range(N):
    for j in range(T):
        if i not in x_structural_faults and j not in y_structural_faults and i1 < M.shape[0]:
            M_old[i][j] = M[i1][j1]
            j1+=1
            if j1 == M.shape[1]:
                j1=0
                i1=i1+1
print M_old
M = M_old

[[ 20.98807397 122.153       19.9884     ...  20.9096      17.83251378
   17.54036854]
 [ 12.57807911  89.5488      29.54739866 ...  20.9096      20.89
   18.13904648]
 [ 18.62212417  20.204      122.153      ...  21.9288      18.12322858
   21.9288    ]
 ...
 [ 18.04571103  30.16374397  28.98459688 ...  15.26862217  15.52329115
   19.714     ]
 [ 18.23895245 122.016      122.153      ...  16.02093506  19.4984
   19.4788    ]
 [         nan 122.016      122.153      ...          nan  19.4984
   19.4788    ]]


### Second Phase 

#### Spatial PreInterpolation Function Defination

In [13]:
def spatial_pre_interpolation():
    '''Step 1'''
    '''H is spatial constraint matrix'''
    H = np.ones((N))
    H = np.diag(H)
    print H
    '''step 2'''
    for x in x_structural_faults:
        H[x] = Y[x]
    print "after replacing rows"
    print H
    '''step 3'''    
    if len(x_structural_faults) > 1:
        for x in x_structural_faults:
            H[:,x] = np.zeros(N)
    print "after making columns to zero"
    print H    
    '''Step 4'''
    for i in range(N):
        sum = 0
        for j in H[i]:
            sum += j
        H[i]  = H[i]/sum
    print "after normalizing rows"
    print H
    ''' HXM has to be done next and return M'''
    for i in x_structural_faults:
        M[i,:] = (M[i-1,:]+M[i+1,:])/2
    return M

#### Temporal PreInterpolation Function Defination

In [14]:
def temporal_pre_interpolation():
    '''step 1'''
    G = np.ones((T))
    G = np.diag(G)
    print G
    '''Step 2'''
    for i in y_structural_faults:
        G[:,i] = np.zeros(T)
        G[:,i] = w[:,i]        
    print "after replacing columns"
    print G
    '''Step 3'''    
    if len(y_structural_faults) > 1:
        for x in y_structural_faults:
            G[x] = np.zeros(T)
    print "after making rows to zero"
    print G
    ''' Step 4'''    
    for i in range(T):
        sum = 0
        for j in G[:,i]:
            sum += j
        G[:,i]  = G[:,i]/sum
    print "after normalizing rows"
    print G
    '''MXG has to be done next and return M'''
    for i in y_structural_faults:
        M[:,i] = (M[:,i-1]+M[:,i+1])/2
    return M

#### Step 1: Spatial pre-interpolation,
where the row data with structure faults are replaced with
the datA from neighboring sensors,<br> taking advantage of spatial
correlation

In [15]:
M = spatial_pre_interpolation()

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
after replacing rows
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
after making columns to zero
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
after normalizing rows
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [16]:
print M

[[21.10787334 12.77494738 19.14062826 ... 17.95       18.93
  18.5282    ]
 [17.7246     18.1628     18.7634     ... 15.23913243 15.70285672
   9.24640597]
 [13.47641144 18.7928     18.2146     ... 13.72366387 14.5869764
  16.44718907]
 ...
 [17.4306     17.4012     16.9798     ... 20.2236     17.0484
  18.9104    ]
 [15.2452     19.2142     19.126      ... 14.50742559 15.8626
  16.6466    ]
 [15.7254     15.6764     22.7226     ... 22.0828     19.714
  19.4788    ]]


#### Step 2: Temporal pre-interpolation,
where the column data experiencing structure faults are replaced
with the data from adjacent time slots,<br> taking advantage of
temporal stability

In [17]:
M = temporal_pre_interpolation()

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
after replacing columns
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
after making rows to zero
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
after normalizing rows
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


#### Step 3: Matrix re-processing,
where matrix completion is applied to recover the original
data matrix X<sub>N×T</sub> from the matrix<br> obtained following above
two procedures, exploiting both spatial correlation and temporal
stability.

In [18]:
#M = NuclearNormMinimization().complete(M)
#M = KNN(k=3).complete(M)
if np.isnan(np.min(M)):
    M =  SoftImpute().complete(M)

In [19]:
print M

[[21.10787334 12.77494738 19.14062826 ... 17.95       18.93
  18.5282    ]
 [17.7246     18.1628     18.7634     ... 15.23913243 15.70285672
   9.24640597]
 [13.47641144 18.7928     18.2146     ... 13.72366387 14.5869764
  16.44718907]
 ...
 [17.4306     17.4012     16.9798     ... 20.2236     17.0484
  18.9104    ]
 [15.2452     19.2142     19.126      ... 14.50742559 15.8626
  16.6466    ]
 [15.7254     15.6764     22.7226     ... 22.0828     19.714
  19.4788    ]]


#### Author: Akash Kumar Verman